# 🎯 Planning & Design Patterns with GitHub Models (.NET)

## 📋 Learning Objectives

This notebook demonstrates enterprise-grade planning and design patterns for building intelligent agents using the Microsoft Agent Framework in .NET with GitHub Models. You'll learn to create agents that can decompose complex problems, plan multi-step solutions, and execute sophisticated workflows with .NET's enterprise features.


## ⚙️ Prerequisites & Setup

**Development Environment:**
- .NET 9.0 SDK or higher
- Visual Studio 2022 or VS Code with C# extension
- GitHub Models API access

**Required Dependencies:**
```xml
<PackageReference Include="Microsoft.Extensions.AI" Version="9.9.0" />
<PackageReference Include="Microsoft.Extensions.AI.OpenAI" Version="9.9.0-preview.1.25458.4" />
<PackageReference Include="DotNetEnv" Version="3.1.1" />
```

**Environment Configuration (.env file):**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```


In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.1"

Installed Packages Microsoft.Extensions.AI, 9.9.1

In [2]:
// #r "nuget: OpenAI, 2.4.0"

In [3]:
#r "nuget: Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.3"

Installed Packages Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.3

In [4]:
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.251001.3"

Installed Packages microsoft.agents.ai, 1.0.0-preview.251001.3

In [5]:
//

In [6]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [7]:
using System;
using System.ClientModel;
using System.Text.Json;
using System.Text.Json.Serialization;
using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using OpenAI;


In [8]:
 using DotNetEnv;

In [9]:
Env.Load("../../../.env");

In [10]:
System.Environment.GetEnvironmentVariable("GITHUB_ENDPOINT");

In [11]:
var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [12]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint= new Uri(github_endpoint)
};

In [13]:


var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [14]:
const string AGENT_NAME ="TravelPlanAgent";

const string AGENT_INSTRUCTIONS = @"You are an planner agent.
    Your job is to decide which agents to run based on the user's request.
    Below are the available agents specialised in different tasks:
    - FlightBooking: For booking flights and providing flight information
    - HotelBooking: For booking hotels and providing hotel information
    - CarRental: For booking cars and providing car rental information
    - ActivitiesBooking: For booking activities and providing activity information
    - DestinationInfo: For providing information about destinations
    - DefaultAgent: For handling general request";

In [15]:

    public class Plan
    {
        [JsonPropertyName("assigned_agent")]
        public string? Assigned_agent { get; set; }

        [JsonPropertyName("task_details")]
        public string? Task_details { get; set; }
    }

In [16]:

    public class TravelPlan
    {
        [JsonPropertyName("main_task")]
        public string? Main_task { get; set; }

        [JsonPropertyName("subtasks")]
        public IList<Plan> Subtasks { get; set; }

    }

In [17]:

ChatClientAgentOptions agentOptions = new(name: AGENT_NAME, instructions: AGENT_INSTRUCTIONS)
{
    ChatOptions = new()
    {
        ResponseFormat = ChatResponseFormatJson.ForJsonSchema(
            schema: AIJsonUtilities.CreateJsonSchema(typeof(TravelPlan)),
            schemaName: "TravelPlan",
            schemaDescription: "Travel Plan with main_task and subtasks")
    }
};

In [18]:
AIAgent agent = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions).GetChatClient(github_model_id).CreateAIAgent(
    agentOptions);

In [19]:

Console.WriteLine(await agent.RunAsync("Create a travel plan for a family of 4, with 2 kids, from Singapore to Melbourne"));

{"main_task":"Plan family trip from Singapore to Melbourne","subtasks":[{"assigned_agent":"FlightBooking","task_details":"Book round-trip flights from Singapore to Melbourne for a family of 4."},{"assigned_agent":"HotelBooking","task_details":"Find and book family-friendly accommodation in Melbourne for the duration of the stay."},{"assigned_agent":"CarRental","task_details":"Arrange car rental in Melbourne for family excursions."},{"assigned_agent":"ActivitiesBooking","task_details":"Research and book family-friendly activities and attractions in Melbourne."},{"assigned_agent":"DestinationInfo","task_details":"Provide information about Melbourne's attractions, dining options, and family-friendly locations."}]}
